# Designing Instagram Feed

A feed is a constantly updating scrollable list of posts, photos, videos, and status updates from all the people and pages a user follows.


## 1. Requirements and System Goals

### Functional requirements
1. Feeds may contain images, videos and text.
2. Feeds are generated from the posts belonging to the pages and people the user follows.
3. The service should support appending new posts as they arrive to the feed for all active users

### Non-functional requirements 
1. The system should be able to generate a user's newsfeed in real-time - maximum latency seen by the end user shoudl be about 2s.

## 2. Capacity Estimation and Constraints
Assume on average a user has 300 friends and follows 200 pages.

**Traffic Estimates:** A typical user checks their feed about 5 times per day on average. If we have 200 million daily active users, then:

```text
Per day: 200M * 5 => 1B requests

Per second:   1B / 86400 sec => 11500 reqs/sec 
```

**Storage estimates:** Assume we have on average 500 posts for each user's feed that we want to keep in memory for fast fetching. 
Let's also assume on average that each post would be about 100KB in size (gifs, photos and videos combined). This means we need about 100KB X 500 = 5 MB per user. 
To store all this data for all active users, we'll need:

```text
    200 M active users * 5MB  ~= 1 Petabyte of memory
```

If a server can hold 100 GB memory, we'd need about 10000 machines to keep the top 500 posts in memory.


## 3. System APIs
> By defining the system APIs, you are explicitly stating what is expected from the system

We'll have REST APIs to expose our service's functionality.

**Getting User Feed**

```python
getUserFeed(
    api_dev_key: int,  # Key of a registered user, used to throttle users based on their allocated quota.
    user_id: int,  # The Id of the user whom the system will generate the feed.
    since_id: int,  # (Optional) Return results with IDs more recent than this ID.
    count: int ,   # (Optional) Specifies number of feed items to try and retrieve.
    max_id: int,   # (Optional) Returns results with IDs younger than the specified ID.
    exclude_replies  # (Optional) Prevents replies from appearing in the results.
```

**Returns:** (JSON) object containing a list of feed items. 